# Datasets: Lead Titanate and Lead Zirconate Titanate (PTO and PZT)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pycroscopy/DTMicroscope/blob/main/data/PZT_Data.ipynb)


This notebook explains how to access several available datasets pertaining to ferroelectric thin films of PZT and PTO.

We will show you where to find the data, how to register it on the digital microscope, and how to explore it offline.

**This notebook will be continuously updated with more datasets in due course, before the hackathon begins.**

By: Rama Vasudevan



# 1. Run server

In [ ]:
!pip install pyro5
!pip install scifireaders
!pip install sidpy
!pip install pynsid
!pip install git+https://github.com/pycroscopy/DTMicroscope.git

In [ ]:
!run_server_afm

# 2. Client side

In [3]:
import matplotlib.pylab as plt
import numpy as np
import Pyro5.api
from IPython.display import clear_output, display

## 2.1 Download Imaging Data

We can clone the repository to get the data files associated with PZT/PTO. This dataset consists of different Piezoresponse Force Microscopy (PFM) Images

In [ ]:
!git clone https://github.com/pycroscopy/DTMicroscope.git

import os
os.listdir('DTMicroscope/data/AFM') #list available images

## 2.2 Register Data to the Digital twin

In [ ]:
uri = "PYRO:microscope.server@localhost:9092" #port for the AFM DT 9092
mic_server = Pyro5.api.Proxy(uri)
import sys

if 'google.colab' in sys.modules:
    mic_server.initialize_microscope("AFM", data_path = r"DTMicroscope/data/AFM/PZT_final0001.h5")
else:
    mic_server.initialize_microscope("AFM", data_path = "../test/datasets/dset_spm1.h5")
mic_server.setup_microscope(data_source = 'Compound_Dataset_1')#upload dataset?
mic_server.get_dataset_info()

# 3. Basic AFM operations

## 3.1 .get_scan()

The `get_scan()` function retrieves scan data from the dataset, with the option to filter by specific channels. Use the `channels` attribute to specify desired channels, allowing you to query particular signals or data channels. For details on available channels, refer to `.get_dataset_info()`.

The `direction` parameter specifies the fast-scanning axis, while the `trace` parameter emulates data acquisition on either the forward or backward pass. These settings are essential for accurately simulating the behavior of the real PID during scanning.

The `modification` parameter allows the application of various imperfections to simulate realistic scanning conditions, such as the effects of a broken tip or non-ideal PID performance.

In [ ]:
#scan without modification
array_list, shape, dtype  =  mic_server.get_scan(channels=['HeightRetrace', 'Phase1Retrace'], 
                                                 modification=None,
                                                 direction = 'horizontal',
                                                 trace = 'forward')
dat =  np.array(array_list, dtype=dtype).reshape(shape)
dat.shape

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7,3))

ax[0].imshow(dat[0].T, cmap='cividis', origin='lower')
ax[0].set_title('HeightRetrace')
ax[1].imshow(dat[1].T, cmap='cividis', origin='lower')
ax[1].set_title('Phase1Retrace')